<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled168.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit-aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 128.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.7 MB/s eta 0:00:00


In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import pickle

# ===== New & Required Imports =====
from datetime import datetime
from google.colab import output
from tqdm.notebook import tqdm

# ===== QISKIT IMPORTS =====
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# ===== GOOGLE DRIVE MOUNT =====
from google.colab import drive
drive.mount('/content/drive')


# ===== Quantum Random Number Generator Class (Used for Seeding) =====
class QuantumRandomGenerator:
    def __init__(self, num_bits_precision=64):
        self.simulator = AerSimulator()
        print("QuantumRandomGenerator configured to use CPU for seeding.")
        self.num_bits = num_bits_precision

    def get_seed(self):
        """Generates a single, high-quality random integer seed."""
        print("Generating a quantum seed... (This may take a moment)")
        circuit = QuantumCircuit(self.num_bits, self.num_bits)
        circuit.h(range(self.num_bits))
        circuit.measure(range(self.num_bits), range(self.num_bits))
        transpiled_circuit = transpile(circuit, self.simulator)
        job = self.simulator.run(transpiled_circuit, shots=1)
        result = job.result()
        counts = result.get_counts(0)
        bit_string = list(counts.keys())[0]
        seed = int(bit_string, 2)
        print(f"Quantum seed generated: {seed}")
        return seed

# --- 1. Environment Class ---
class Environment:
    def __init__(self, rng, days_per_cycle=50):
        self.days_per_cycle, self.time, self.rng = days_per_cycle, 0, rng
        self.steps_per_year = 365 * self.days_per_cycle
        self.uv_intensity, self.temperature, self.yearly_uv_severity = self._generate_cycles(num_years=50)

    def _generate_cycles(self, num_years):
        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, self.days_per_cycle)); daily_uv_cycle[daily_uv_cycle < 0] = 0
        daily_temp_swing = -4 * np.cos(np.linspace(0, 2 * np.pi, self.days_per_cycle))
        full_uv, full_temp, yearly_severity_log = [], [], []

        for _ in range(num_years):
            uv_severity = self.rng.uniform(0.6, 1.4)
            yearly_severity_log.extend([uv_severity] * self.steps_per_year)
            seasonal_cycle = np.sin(np.linspace(0, 2 * np.pi, self.steps_per_year))
            seasonal_temp_base = 15 - 10 * np.cos(np.linspace(0, 2 * np.pi, self.steps_per_year))
            seasonal_amplitude_mod = (0.225 * seasonal_cycle + 0.725) * uv_severity
            GEOTHERMAL_BASE_TEMP = 4.0
            year_temp = []

            for i in range(self.steps_per_year):
                cloud_cover_factor = self.rng.uniform(0.7, 1.0)
                solar_temp = seasonal_temp_base[i] + daily_temp_swing[i % self.days_per_cycle]
                year_temp.append(max(GEOTHERMAL_BASE_TEMP, solar_temp))
                daily_uv = daily_uv_cycle[i % self.days_per_cycle]
                seasonal_uv = daily_uv * seasonal_amplitude_mod[i]
                final_uv = seasonal_uv * cloud_cover_factor
                full_uv.append(final_uv)

            full_temp.extend(year_temp)
        return np.array(full_uv), np.array(full_temp), np.array(yearly_severity_log)

    def get_current_uv(self): return self.uv_intensity[self.time % len(self.uv_intensity)]
    def get_current_temperature(self): return self.temperature[self.time % len(self.temperature)]
    def get_current_uv_severity(self): return self.yearly_uv_severity[self.time % len(self.yearly_uv_severity)]
    def step(self): self.time += 1

# --- 2. Protoribosome Class ---
class Protoribosome:
    def __init__(self, env, strategy, initial_sequence, rng, initial_mass=100.0):
        self.env, self.strategy, self.rna_sequence, self.rng = env, strategy, list(initial_sequence), rng
        self.rna_mass, self.atp_pool, self.mcyw_peptide_pool, self.other_peptide_pool = initial_mass, 500.0, 50.0, 5.0
        self.rna_damage_level, self.location, self.status, self.metabolic_efficiency = 0.0, 'shadow_zone', 'ACTIVE', 1.0
        self.uv_damage_rate_per_uv, self.repair_rate = 1.2, 0.5
        self.uv_protection_factor, self.damage_tolerance_threshold = 0.01, 3.0
        self.atp_generation_rate_per_mcyw_uv, self.baseline_atp_chemosynthesis, self.protein_synthesis_rate = 0.15, 1.5, 0.20
        self.replication_rate, self.atp_degradation_rate, self.mutation_prob = 0.05, 0.025, 0.004
        self.growth_rate, self.optimal_temp, self.temp_tolerance = 0.1, 25.0, 15.0
    def get_peptide_type(self):
        codons = ["".join(self.rna_sequence[i:i+3]) for i in range(0, len(self.rna_sequence), 3)]
        return "MCYW" if any(c in {'UGU', 'UGC', 'UAU', 'UAC', 'UGG'} for c in codons) else "XXXX"
    def has_stop_codon(self): return any(c in {'UAA', 'UAG', 'UGA'} for c in ["".join(self.rna_sequence[i:i+3]) for i in range(0, len(self.rna_sequence), 3)])
    def _decide_zone_transition(self):
        if self.location == 'shadow_zone' and self.atp_pool < 450: self.location = 'sunlight_zone'
        elif self.location == 'sunlight_zone' and self.rna_damage_level > self.damage_tolerance_threshold * 0.75: self.location = 'shadow_zone'
    def _apply_uv_damage(self, current_uv):
        if self.location == 'shadow_zone' or current_uv <= 0:
            self.rna_damage_level = max(0, self.rna_damage_level - self.repair_rate * 0.1)
            return
        protection = self.mcyw_peptide_pool * self.uv_protection_factor
        effective_uv = current_uv * max(0.01, 1 - protection)
        self.rna_damage_level += effective_uv * self.uv_damage_rate_per_uv
        for i in range(len(self.rna_sequence)):
            if self.rng.random() < self.mutation_prob * effective_uv: self.rna_sequence[i] = self.rng.choice(list("AUGC"))
    def _manage_state(self, has_stop_codon_cached):
        if self.atp_pool < 0.1 or self.rna_damage_level >= self.damage_tolerance_threshold: self.status = 'INACTIVE'; return
        if self.strategy == 'cautious' and has_stop_codon_cached: self.status = 'ARRESTED'
        elif self.status == 'ARRESTED' and not has_stop_codon_cached: self.status = 'ACTIVE'
        self.metabolic_efficiency = 1.25 if self.status == 'ACTIVE' else 1.0
    def _synthesize_protein(self, has_stop_codon_cached, temp_factor):
        if self.status != 'ACTIVE': return
        if self.strategy == 'readthrough' and has_stop_codon_cached: self.status = 'INACTIVE'; return
        units = self.protein_synthesis_rate * self.metabolic_efficiency * temp_factor * 10
        if self.get_peptide_type() == "MCYW": self.mcyw_peptide_pool += units
        else: self.other_peptide_pool += units
        self.atp_pool -= units * 0.02
    def _replicate_rna(self, temp_factor):
        if self.status != 'ACTIVE' or self.rna_mass < 80: return None
        can_replicate = self.rna_mass * self.replication_rate * self.metabolic_efficiency * temp_factor
        if self.atp_pool > can_replicate * 0.2:
            offspring_mass = self.rna_mass * 0.5
            self.rna_mass -= offspring_mass
            self.atp_pool -= can_replicate * 0.2
            return Protoribosome(self.env, self.strategy, "".join(self.rna_sequence), self.rng, offspring_mass)
        return None
    def _repair_rna_dna(self, dna_template, temp_factor):
        if self.strategy == 'readthrough': return
        repair_efficiency = temp_factor
        if self.status == 'ARRESTED': repair_efficiency *= 0.2
        elif self.status != 'ACTIVE': return
        if "".join(self.rna_sequence) != dna_template:
            for i in range(len(self.rna_sequence)):
                if self.rna_sequence[i] != dna_template[i]:
                    if self.rng.random() < repair_efficiency:
                        if self.atp_pool > 0.0025:
                            self.atp_pool -= 0.0025; self.rna_sequence[i] = dna_template[i]
                            self.rna_damage_level = max(0, self.rna_damage_level - self.repair_rate)
                    break
    def _grow_mass(self, temp_factor):
        if self.status != 'ACTIVE': return
        potential_growth = self.rna_mass * self.growth_rate * self.metabolic_efficiency * temp_factor
        self.rna_mass += potential_growth
        self.atp_pool -= potential_growth * 0.1
    def step(self, current_uv, current_temp, dna_template):
        if self.status == 'INACTIVE': return None
        if current_temp < 10: temp_factor = 0.2 + ((current_temp - 5) / 5.0) * 0.8; temp_factor = max(0.2, temp_factor)
        elif current_temp > 22: heat_penalty = (current_temp - 22) / 6.0; temp_factor = max(0.1, 1.0 - heat_penalty)
        else: temp_factor = 1.0
        self._decide_zone_transition(); self._apply_uv_damage(current_uv); self._repair_rna_dna(dna_template, temp_factor)
        has_stop = self.has_stop_codon()
        self._manage_state(has_stop)
        if self.status == 'INACTIVE': return None
        self.atp_pool += self.baseline_atp_chemosynthesis; self.atp_pool *= (1 - self.atp_degradation_rate)
        if self.location == 'sunlight_zone' and current_uv > 0:
            photosynthesis_gain = self.atp_generation_rate_per_mcyw_uv * self.mcyw_peptide_pool * current_uv
            if current_temp > 18: photosynthesis_gain *= 1.5
            self.atp_pool += photosynthesis_gain
        self._grow_mass(temp_factor); self. _synthesize_protein(has_stop, temp_factor)
        if self.status == 'INACTIVE': return None
        return self._replicate_rna(temp_factor)

# --- 3. Colony Class ---
class Colony:
    def __init__(self, env, dna_template, initial_pop_size, steps_per_day, rng, shuffle_rng):
        self.env, self.dna_template, self.steps_per_day, self.max_population = env, dna_template, steps_per_day, 10000
        self.rng, self.shuffle_rng = rng, shuffle_rng
        self.active_population = [Protoribosome(env, 'cautious' if i%2==0 else 'readthrough', dna_template, rng) for i in range(initial_pop_size)]
        self.inactive_population = []
    def step(self, current_step):
        current_uv, current_temp = self.env.get_current_uv(), self.env.get_current_temperature()
        if current_step > 0 and current_step % self.steps_per_day == 0: self.inactive_population.clear()
        next_generation = []
        for protoribosome in self.active_population:
            new_offspring = protoribosome.step(current_uv, current_temp, self.dna_template)
            if protoribosome.status != 'INACTIVE': next_generation.append(protoribosome)
            else: self.inactive_population.append(protoribosome)
            if new_offspring: next_generation.append(new_offspring)
        self.active_population = next_generation
        if len(self.active_population) > self.max_population:
            self.shuffle_rng.shuffle(self.active_population)
            self.active_population = self.active_population[:self.max_population]

    def get_aggregated_data(self):
        data = { 'cautious': 0, 'readthrough': 0, 'arrested': 0, 'inactive': 0, 'in_sunlight_zone': 0,
                 'cautious_avg_damage': 0.0, 'readthrough_avg_damage': 0.0 }
        if not self.active_population:
            data['inactive'] = len(self.inactive_population)
            return data
        for p in self.active_population:
            if p.strategy == 'cautious':
                data['cautious'] += 1
                data['cautious_avg_damage'] += p.rna_damage_level
                if p.status == 'ARRESTED': data['arrested'] += 1
            else:
                data['readthrough'] += 1
                data['readthrough_avg_damage'] += p.rna_damage_level
            if p.location == 'sunlight_zone': data['in_sunlight_zone'] += 1
        if data['cautious'] > 0: data['cautious_avg_damage'] /= data['cautious']
        if data['readthrough'] > 0: data['readthrough_avg_damage'] /= data['readthrough']
        data['inactive'] = len(self.inactive_population)
        return data

    def get_genetic_load_analysis(self):
        cautious_total, cautious_with_stop, reckless_total, reckless_with_stop = 0, 0, 0, 0
        for p in self.active_population:
            has_stop = p.has_stop_codon()
            if p.strategy == 'cautious':
                cautious_total += 1
                if has_stop: cautious_with_stop += 1
            else:
                reckless_total += 1
                if has_stop: reckless_with_stop += 1
        cautious_stop_pct = (cautious_with_stop / cautious_total * 100) if cautious_total > 0 else 0
        reckless_stop_pct = (reckless_with_stop / reckless_total * 100) if reckless_total > 0 else 0
        return cautious_total, cautious_with_stop, cautious_stop_pct, reckless_total, reckless_with_stop, reckless_stop_pct

# --- 4. Simulation Runner ---
def run_single_simulation(steps_per_day, rng, shuffle_rng, save_path):
    state_file_path = save_path + ".pkl"
    if os.path.exists(state_file_path):
        print(f"--- Resuming simulation from state file... ---")
        with open(state_file_path, 'rb') as f: saved_state = pickle.load(f)
        colony, start_step, data_log = saved_state['colony'], saved_state['step_count'], saved_state['log']
        colony.env.time = start_step
    else:
        print(f"\n--- Starting New Replica ---")
        start_step = 0
        data_log = []
        IDEAL_DNA_TEMPLATE = "AUGUGUUACUGG"
        env = Environment(rng, days_per_cycle=steps_per_day)
        colony = Colony(env, IDEAL_DNA_TEMPLATE, initial_pop_size=5000, steps_per_day=steps_per_day, rng=rng, shuffle_rng=shuffle_rng)

    progress_bar = tqdm(desc="Simulating Replica", initial=start_step, total=float('inf'), leave=False)
    step_count = start_step
    winner = "Unknown"
    postfix_dict = {'Status': 'Running...'}
    progress_bar.set_postfix(postfix_dict)

    while True:
        cautious_count = colony.get_aggregated_data().get('cautious', 0)
        readthrough_count = colony.get_aggregated_data().get('readthrough', 0)

        if step_count > 1000:
            if cautious_count == 0 and readthrough_count == 0: winner = "Mutual Extinction"; break
            elif cautious_count == 0: winner = "Reckless"; break
            elif readthrough_count == 0: winner = "Cautious"; break

        colony.step(step_count)
        agg_data = colony.get_aggregated_data()
        c_tot, c_stop_ct, c_stop_pct, r_tot, r_stop_ct, r_stop_pct = colony.get_genetic_load_analysis()
        log_entry = { 'time': colony.env.time, 'uv': colony.env.get_current_uv(), 'temp': colony.env.get_current_temperature(),
                      'uv_severity': colony.env.get_current_uv_severity(), **agg_data,
                      'cautious_stop_pct': c_stop_pct, 'reckless_stop_pct': r_stop_pct }
        data_log.append(log_entry)
        colony.env.step()
        progress_bar.update(1)

        if step_count > 0 and step_count % 500 == 0:
            temp_save_path = state_file_path + ".tmp"
            state_to_save = {'step_count': step_count, 'colony': colony, 'log': data_log}
            with open(temp_save_path, 'wb') as f: pickle.dump(state_to_save, f)
            os.replace(temp_save_path, state_file_path)

        step_count += 1

    progress_bar.close()
    final_day = (step_count -1) // steps_per_day
    print(f"\n--- KNOCKOUT on day {final_day}: {winner} wins. ---")
    results_df = pd.DataFrame(data_log)
    summary = { 'Winner': winner, 'Duration (Days)': final_day,
                'Final Cautious': colony.get_aggregated_data().get('cautious', 0),
                'Final Reckless': colony.get_aggregated_data().get('readthrough', 0) }

    # Save final results dataframe and remove the state file
    results_df.to_pickle(save_path + ".df.pkl")
    if os.path.exists(state_file_path):
        os.remove(state_file_path)

    return summary, results_df

# --- 5. Plotting Function ---
def plot_simulation_details(results_df, replica_title=""):
    if results_df.empty:
        print("No data to plot.")
        return
    fig, axs = plt.subplots(4, 1, figsize=(15, 18), sharex=True, gridspec_kw={'hspace': 0.4})
    fig.suptitle(f'Simulation Detailed Results: {replica_title}', fontsize=16)
    colors = {'cautious': 'orange', 'readthrough': 'purple', 'inactive': 'grey'}

    # Plot 1: Environmental Conditions
    ax0_twin = axs[0].twinx()
    axs[0].plot(results_df['time'], results_df['uv'], color='lightgray', alpha=0.7, label='Daily UV', lw=0.5)
    ax0_twin.plot(results_df['time'], results_df['temp'], color='lightcoral', alpha=0.6, label='Daily Temp (°C)', lw=0.5)
    rolling_window = 10 * 10
    axs[0].plot(results_df['time'], results_df['uv'].rolling(window=rolling_window, min_periods=1).mean(), color='black', label=f'UV Trend ({rolling_window//10}-day avg)', lw=1.5)
    ax0_twin.plot(results_df['time'], results_df['temp'].rolling(window=rolling_window, min_periods=1).mean(), color='red', label=f'Temp Trend ({rolling_window//10}-day avg)', lw=1.5)
    axs[0].set_title('Environmental Conditions'); axs[0].set_ylabel('UV Intensity'); ax0_twin.set_ylabel('Temperature (°C)', color='r')
    axs[0].legend(loc='upper left'); ax0_twin.legend(loc='upper right')

    # Plot 2: Active Population
    axs[1].plot(results_df['time'], results_df['cautious'], label='Cautious', color=colors['cautious'])
    axs[1].plot(results_df['time'], results_df['readthrough'], label='Reckless', color=colors['readthrough'])
    axs[1].set_title('Active Population by Strategy'); axs[1].set_ylabel('Count'); axs[1].legend(); axs[1].set_yscale('linear')

    # Plot 3: Population Composition
    axs[2].stackplot(results_df['time'], results_df['cautious'], results_df['readthrough'], results_df['inactive'],
                     labels=['Cautious', 'Reckless', 'Inactive'], colors=[colors['cautious'], colors['readthrough'], colors['inactive']])
    axs[2].set_title('Total Population Composition'); axs[2].set_ylabel('Total Count'); axs[2].legend(loc='upper left')

    # Plot 4: Average RNA Damage per Population
    axs[3].plot(results_df['time'], results_df['cautious_avg_damage'], label='Cautious Avg. Damage', color=colors['cautious'])
    axs[3].plot(results_df['time'], results_df['readthrough_avg_damage'], label='Reckless Avg. Damage', color=colors['readthrough'], linestyle='--')
    axs[3].set_title('Average RNA Damage per Population'); axs[3].set_ylabel('Average Damage Level'); axs[3].legend(loc='upper left')

    for ax in axs: ax.grid(True, linestyle=':', linewidth='0.5', color='gray')
    if not results_df.empty:
        axs[-1].set_xlabel("Time (Simulation Steps)", fontsize=12); axs[-1].set_xlim(left=0, right=results_df['time'].max())
    plt.tight_layout(rect=[0, 0, 1, 0.96]); plt.show()


# --- 6. Main Orchestrator ---
def main_orchestrator(action, num_replicas, steps_per_day):
    BASE_SAVE_DIR = os.path.join('/content/drive/My Drive/Colab Notebooks/Sim_Experiments/')
    os.makedirs(BASE_SAVE_DIR, exist_ok=True)
    all_runs = sorted([d for d in os.listdir(BASE_SAVE_DIR) if os.path.isdir(os.path.join(BASE_SAVE_DIR, d))])

    experiment_dir = None
    if action == 'fresh_start':
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        experiment_dir = os.path.join(BASE_SAVE_DIR, f"Experiment_{timestamp}")
        print(f"--- FRESH START selected. Creating new experiment directory: {os.path.basename(experiment_dir)} ---")
        os.makedirs(experiment_dir, exist_ok=True)

    elif action in ('resume', 'plot_only', 'monitor'):
        if not all_runs:
            print(f"--- {action.upper()} selected, but no past experiments found. Please run a 'fresh_start' first. ---")
            return
        experiment_dir = os.path.join(BASE_SAVE_DIR, all_runs[-1])
        print(f"--- {action.upper()} selected. Using latest experiment: {os.path.basename(experiment_dir)} ---")

    if not experiment_dir:
        print("Could not determine experiment directory. Exiting.")
        return

    # --- Plotting/Monitoring Action ---
    if action in ('plot_only', 'monitor'):
        last_completed_replica_path = None
        for i in range(num_replicas, 0, -1):
            replica_result_path = os.path.join(experiment_dir, f"Replica_{i}", "results.df.pkl")
            if os.path.exists(replica_result_path):
                last_completed_replica_path = replica_result_path
                break
        if last_completed_replica_path:
            print(f"Loading results from {os.path.dirname(last_completed_replica_path)}")
            results_df = pd.read_pickle(last_completed_replica_path)
            plot_simulation_details(results_df, replica_title=os.path.basename(os.path.dirname(last_completed_replica_path)))
        else:
            print("No completed replica results found to plot in this experiment.")
        return

    # --- Simulation Actions (Fresh Start / Resume) ---
    all_summaries = []
    for i in range(1, num_replicas + 1):
        replica_dir = os.path.join(experiment_dir, f"Replica_{i}")
        os.makedirs(replica_dir, exist_ok=True)
        replica_save_base = os.path.join(replica_dir, "simulation")
        final_results_path = replica_save_base + ".df.pkl"

        print("\n" + "="*80)
        print(f"Processing Replica {i}/{num_replicas}")
        print("="*80)

        if os.path.exists(final_results_path):
            print("Replica already completed. Loading summary and skipping simulation.")
            results_df = pd.read_pickle(final_results_path)
            last_row = results_df.iloc[-1]
            winner = "Cautious" if last_row['readthrough'] == 0 else "Reckless" if last_row['cautious'] == 0 else "Mutual Extinction"
            summary = {'Replica': i, 'Winner': winner, 'Duration (Days)': (last_row['time']-1)//steps_per_day,
                       'Final Cautious': last_row['cautious'], 'Final Reckless': last_row['readthrough']}
            all_summaries.append(summary)
            plot_simulation_details(results_df, replica_title=f"Replica {i} (Loaded)")
            continue

        seed_file_path = os.path.join(replica_dir, 'quantum_seed.txt')
        if not os.path.exists(seed_file_path):
            q_rng = QuantumRandomGenerator(num_bits_precision=29)
            quantum_seed = q_rng.get_seed()
            with open(seed_file_path, 'w') as f: f.write(str(quantum_seed))
        else:
            with open(seed_file_path, 'r') as f: quantum_seed = int(f.read())

        shuffle_rng = random.Random(quantum_seed)
        sim_rng = np.random.default_rng(seed=quantum_seed)

        summary, details_df = run_single_simulation(
            steps_per_day=steps_per_day, rng=sim_rng,
            shuffle_rng=shuffle_rng, save_path=replica_save_base
        )
        summary['Replica'] = i
        all_summaries.append(summary)

        # Plot results at the end of each replica run
        plot_simulation_details(details_df, replica_title=f"Replica {i}")

    # --- Final Summary Report ---
    print("\n\n" + "="*80)
    print(" EXPERIMENT COMPLETE: FINAL SUMMARY ".center(80, "="))
    print("="*80)
    if all_summaries:
        summary_df = pd.DataFrame(all_summaries)
        print(summary_df.to_string())
    else:
        print("No simulations were run to summarize.")
    print("="*80)


if __name__ == "__main__":
    # =======================================================
    # ===      CHOOSE YOUR ACTION AND SETTINGS HERE       ===
    # =======================================================
    # Options:
    # 'fresh_start': Run a new experiment with N replicas.
    # 'resume':      Continue the most recent experiment.
    # 'plot_only':   Plot the results of the last completed replica.
    # 'monitor':     Alias for 'plot_only'.

    CHOSEN_ACTION = 'fresh_start'
    NUM_REPLICAS = 3   # Set the number of replicas to run (e.g., 10)
    STEPS_PER_DAY = 10 # Timesteps per day in the simulation

    # =======================================================

    main_orchestrator(
        action=CHOSEN_ACTION,
        num_replicas=NUM_REPLICAS,
        steps_per_day=STEPS_PER_DAY
    )



ModuleNotFoundError: No module named 'qiskit'